# Classifying Urban sounds using Deep Learning

## 3 Model Training and Evaluation 

### Load Preprocessed data 

In [1]:
# retrieve the preprocessed data from previous notebook

%store -r x_train 
%store -r x_test 
%store -r y_train 
%store -r y_test 
%store -r yy 
%store -r le

### Initial model architecture - MLP

We will start with constructing a Multilayer Perceptron (MLP) Neural Network using Keras and a Tensorflow backend. 

Starting with a `sequential` model so we can build the model layer by layer. 

We will begin with a simple model architecture, consisting of three layers, an input layer, a hidden layer and an output layer. All three layers will be of the `dense` layer type which is a standard layer type that is used in many cases for neural networks. 

The first layer will receive the input shape. As each sample contains 40 MFCCs (or columns) we have a shape of (1x40) this means we will start with an input shape of 40. 

The first two layers will have 256 nodes. The activation function we will be using for our first 2 layers is the `ReLU`, or `Rectified Linear Activation`. This activation function has been proven to work well in neural networks.

We will also apply a `Dropout` value of 50% on our first two layers. This will randomly exclude nodes from each update cycle which in turn results in a network that is capable of better generalisation and is less likely to overfit the training data.

Our output layer will have 10 nodes (num_labels) which matches the number of possible classifications. The activation is for our output layer is `softmax`. Softmax makes the output sum up to 1 so the output can be interpreted as probabilities. The model will then make its prediction based on which option has the highest probability.

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()

model.add(Dense(256, input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

Using TensorFlow backend.


### Compiling the model 

For compiling our model, we will use the following three parameters: 

* Loss function - we will use `categorical_crossentropy`. This is the most common choice for classification. A lower score indicates that the model is performing better.

* Metrics - we will use the `accuracy` metric which will allow us to view the accuracy score on the validation data when we train the model. 

* Optimizer - here we will use `adam` which is a generally good optimizer for many use cases.


In [3]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam') 

In [4]:
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               10496     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)               

### Training 

Here we will train the model. 

We will start with 100 epochs which is the number of times the model will cycle through the data. The model will improve on each cycle until it reaches a certain point. 

We will also start with a low batch size, as having a large batch size can reduce the generalisation ability of the model. 

In [5]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 

num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Train on 6985 samples, validate on 1747 samples
Epoch 1/100
6985/6985 [==============================] - 1s 202us/step - loss: 8.6280 - accuracy: 0.1873 - val_loss: 2.1022 - val_accuracy: 0.2999

Epoch 00001: val_loss improved from inf to 2.10219, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 2/100
6985/6985 [==============================] - 1s 144us/step - loss: 2.2520 - accuracy: 0.2311 - val_loss: 1.9448 - val_accuracy: 0.3085

Epoch 00002: val_loss improved from 2.10219 to 1.94476, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 3/100
6985/6985 [==============================] - 1s 156us/step - loss: 2.0282 - accuracy: 0.2737 - val_loss: 1.8046 - val_accuracy: 0.3412

Epoch 00003: val_loss improved from 1.94476 to 1.80456, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 4/100
6985/6985 [==============================] - 1s 165us/step - loss: 1.9008 - accuracy: 0.3213 - val_loss: 1.6954 - val_accuracy: 0.4030

Epoch 00004: val_loss imp

6985/6985 [==============================] - 1s 135us/step - loss: 0.7581 - accuracy: 0.7440 - val_loss: 0.5872 - val_accuracy: 0.8191

Epoch 00032: val_loss improved from 0.59937 to 0.58722, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 33/100
6985/6985 [==============================] - 1s 149us/step - loss: 0.7412 - accuracy: 0.7486 - val_loss: 0.5820 - val_accuracy: 0.8168

Epoch 00033: val_loss improved from 0.58722 to 0.58199, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 34/100
6985/6985 [==============================] - 1s 156us/step - loss: 0.7542 - accuracy: 0.7429 - val_loss: 0.5800 - val_accuracy: 0.8248

Epoch 00034: val_loss improved from 0.58199 to 0.57996, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 35/100
6985/6985 [==============================] - 1s 156us/step - loss: 0.7103 - accuracy: 0.7525 - val_loss: 0.5896 - val_accuracy: 0.8180

Epoch 00035: val_loss did not improve from 0.57996
Epoch 36/100
6985/6985 [===


Epoch 00068: val_loss did not improve from 0.46629
Epoch 69/100
6985/6985 [==============================] - 1s 168us/step - loss: 0.5769 - accuracy: 0.8094 - val_loss: 0.4926 - val_accuracy: 0.8563

Epoch 00069: val_loss did not improve from 0.46629
Epoch 70/100
6985/6985 [==============================] - 1s 159us/step - loss: 0.5937 - accuracy: 0.8024 - val_loss: 0.4757 - val_accuracy: 0.8666

Epoch 00070: val_loss did not improve from 0.46629
Epoch 71/100
6985/6985 [==============================] - 1s 153us/step - loss: 0.5543 - accuracy: 0.8173 - val_loss: 0.4561 - val_accuracy: 0.8666

Epoch 00071: val_loss improved from 0.46629 to 0.45609, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 72/100
6985/6985 [==============================] - 1s 144us/step - loss: 0.5646 - accuracy: 0.8007 - val_loss: 0.4666 - val_accuracy: 0.8666

Epoch 00072: val_loss did not improve from 0.45609
Epoch 73/100
6985/6985 [==============================] - 1s 146us/step - loss: 0.5489

### Test the model 

Here we will review the accuracy of the model on both the training and test data sets. 

In [6]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.9229778051376343
Testing Accuracy:  0.8717802166938782


The initial Training and Testing accuracy scores are quite high. As there is not a great difference between the Training and Test scores (~5%) this suggests that the model has not suffered from overfitting. 

### Predictions  

Here we will build a method which will allow us to test the models predictions on a specified audio .wav file. 

In [7]:
import librosa 
import numpy as np 

def extract_feature(file_name):
   
    try:
        audio_data, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None, None

    return np.array([mfccsscaled])


In [8]:
def print_prediction(file_name):
    prediction_feature = extract_feature(file_name) 

    predicted_vector = model.predict_classes(prediction_feature)
    predicted_class = le.inverse_transform(predicted_vector) 
    print("The predicted class is:", predicted_class[0], '\n') 

    predicted_proba_vector = model.predict_proba(prediction_feature) 
    predicted_proba = predicted_proba_vector[0]
    for i in range(len(predicted_proba)): 
        category = le.inverse_transform(np.array([i]))
        print(category[0], "\t\t : ", format(predicted_proba[i], '.32f') )

### Validation 

#### Test with sample data 

Initial sainity check to verify the predictions using a subsection of the sample audio files we explored in the first notebook. We expect the bulk of these to be classified correctly. 

In [9]:
# Class: Air Conditioner

filename = '../UrbanSound Dataset sample/audio/100852-0-0-0.wav' 
print_prediction(filename) 

The predicted class is: air_conditioner 

air_conditioner 		 :  0.99999511241912841796875000000000
car_horn 		 :  0.00000015479932358175574336200953
children_playing 		 :  0.00000099942099041072651743888855
dog_bark 		 :  0.00000010786862247869066777639091
drilling 		 :  0.00000134932452056091278791427612
engine_idling 		 :  0.00000163115601026220247149467468
gun_shot 		 :  0.00000004490967242531951342243701
jackhammer 		 :  0.00000001620071188312977028544992
siren 		 :  0.00000000126438137737494571410934
street_music 		 :  0.00000076529892112375819124281406


In [10]:
# Class: Drilling

filename = '../UrbanSound Dataset sample/audio/103199-4-0-0.wav'
print_prediction(filename) 

The predicted class is: drilling 

air_conditioner 		 :  0.00000000000000423190831810578254
car_horn 		 :  0.00000015316409474053216399624944
children_playing 		 :  0.00000014709462448081467300653458
dog_bark 		 :  0.00000254344990935351233929395676
drilling 		 :  0.99989640712738037109375000000000
engine_idling 		 :  0.00000000000049928832769607911501
gun_shot 		 :  0.00000000385988752071853014058433
jackhammer 		 :  0.00000000023016226424754648860471
siren 		 :  0.00000000000883451384964262231847
street_music 		 :  0.00010069450217997655272483825684


In [11]:
# Class: Street music 

filename = '../UrbanSound Dataset sample/audio/101848-9-0-0.wav'
print_prediction(filename) 

The predicted class is: street_music 

air_conditioner 		 :  0.09999072551727294921875000000000
car_horn 		 :  0.00305506144650280475616455078125
children_playing 		 :  0.09950152784585952758789062500000
dog_bark 		 :  0.02582867257297039031982421875000
drilling 		 :  0.00509325042366981506347656250000
engine_idling 		 :  0.00916280318051576614379882812500
gun_shot 		 :  0.00549275847151875495910644531250
jackhammer 		 :  0.03270008042454719543457031250000
siren 		 :  0.00361734302714467048645019531250
street_music 		 :  0.71555775403976440429687500000000


In [12]:
# Class: Car Horn 

filename = '../UrbanSound Dataset sample/audio/100648-1-0-0.wav'
print_prediction(filename) 

The predicted class is: car_horn 

air_conditioner 		 :  0.00188611494377255439758300781250
car_horn 		 :  0.68632853031158447265625000000000
children_playing 		 :  0.01224335655570030212402343750000
dog_bark 		 :  0.16461659967899322509765625000000
drilling 		 :  0.05645351111888885498046875000000
engine_idling 		 :  0.00212736334651708602905273437500
gun_shot 		 :  0.00211420282721519470214843750000
jackhammer 		 :  0.00372551172040402889251708984375
siren 		 :  0.00587591761723160743713378906250
street_music 		 :  0.06462877988815307617187500000000


#### Observations 

From this brief sanity check the model seems to predict well. One errror was observed whereby a car horn was incorrectly classifed as a dog bark. 

We can see from the per class confidence that this was quite a low score (43%). This allows follows our early observation that a dog bark and car horn are similar in spectral shape. 

### Other audio

Here we will use a sample of various copyright free sounds that we not part of either our test or training data to further validate our model. 

In [11]:
filename = '../Evaluation audio/dog_bark_1.wav'
print_prediction(filename) 

The predicted class is: dog_bark 

air_conditioner 		 :  0.00001390586567140417173504829407
car_horn 		 :  0.00530895218253135681152343750000
children_playing 		 :  0.39210268855094909667968750000000
dog_bark 		 :  0.44702345132827758789062500000000
drilling 		 :  0.00319736590608954429626464843750
engine_idling 		 :  0.00059610337484627962112426757812
gun_shot 		 :  0.11026044934988021850585937500000
jackhammer 		 :  0.00000000303021985281759498320753
siren 		 :  0.00992829445749521255493164062500
street_music 		 :  0.03156874328851699829101562500000


In [12]:
filename = '../Evaluation audio/drilling_1.wav'

print_prediction(filename) 

The predicted class is: drilling 

air_conditioner 		 :  0.00031856526038609445095062255859
car_horn 		 :  0.00001915698339871596544981002808
children_playing 		 :  0.00003096606451435945928096771240
dog_bark 		 :  0.00005764564048149622976779937744
drilling 		 :  0.97377794981002807617187500000000
engine_idling 		 :  0.00008627835632069036364555358887
gun_shot 		 :  0.00000530346187588293105363845825
jackhammer 		 :  0.02570128440856933593750000000000
siren 		 :  0.00000010822022744605419575236738
street_music 		 :  0.00000267743575932399835437536240


In [13]:
filename = '../Evaluation audio/gun_shot_1.wav'

print_prediction(filename) 

# sample data weighted towards gun shot - peak in the dog barking sample is simmilar in shape to the gun shot sample

The predicted class is: street_music 

air_conditioner 		 :  0.01734401285648345947265625000000
car_horn 		 :  0.00009161393973045051097869873047
children_playing 		 :  0.00024948510690592229366302490234
dog_bark 		 :  0.41893532872200012207031250000000
drilling 		 :  0.00050316314445808529853820800781
engine_idling 		 :  0.00616690935567021369934082031250
gun_shot 		 :  0.00489603541791439056396484375000
jackhammer 		 :  0.00000227677332986786495894193649
siren 		 :  0.00087137427181005477905273437500
street_music 		 :  0.55093985795974731445312500000000


In [16]:
filename = '../Evaluation audio/siren_1.wav'

print_prediction(filename) 

The predicted class is: siren 

air_conditioner 		 :  0.00000732402349967742338776588440
car_horn 		 :  0.00057092373026534914970397949219
children_playing 		 :  0.00199068244546651840209960937500
dog_bark 		 :  0.02090488374233245849609375000000
drilling 		 :  0.00046552356798201799392700195312
engine_idling 		 :  0.14164580404758453369140625000000
gun_shot 		 :  0.00050196843221783638000488281250
jackhammer 		 :  0.00276053301058709621429443359375
siren 		 :  0.81527197360992431640625000000000
street_music 		 :  0.01588040776550769805908203125000


#### Observations 

The performance of our initial model is satisfactorry and has generalised well, seeming to predict well when tested against new audio data. 

### *In the next notebook we will refine our model*